##Odd Even Digit Recognition##

In [32]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [33]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [34]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [tf.keras.layers.Flatten(input_shape=[28,28]),
          tf.keras.layers.Dense(300,kernel_initializer = "he_normal"),
          tf.keras.layers.LeakyReLU(),
          tf.keras.layers.Dense(100,kernel_initializer="he_normal"),
          tf.keras.layers.LeakyReLU(),
          tf.keras.layers.Dense(10,activation="softmax")]

In [35]:
model = tf.keras.models.Sequential(LAYERS)

In [36]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics= ["accuracy"])

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 300)               235500    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
________________________________________________

In [38]:
history = model.fit(X_train,y_train,epochs=10,validation_data=(X_valid,y_valid),verbose=2)

Epoch 1/10
1719/1719 - 5s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980
Epoch 2/10
1719/1719 - 4s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596
Epoch 3/10
1719/1719 - 4s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834
Epoch 4/10
1719/1719 - 4s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940
Epoch 5/10
1719/1719 - 4s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006
Epoch 6/10
1719/1719 - 4s - loss: 0.3852 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052
Epoch 7/10
1719/1719 - 4s - loss: 0.3644 - accuracy: 0.8980 - val_loss: 0.3348 - val_accuracy: 0.9102
Epoch 8/10
1719/1719 - 5s - loss: 0.3485 - accuracy: 0.9021 - val_loss: 0.3209 - val_accuracy: 0.9138
Epoch 9/10
1719/1719 - 4s - loss: 0.3356 - accuracy: 0.9053 - val_loss: 0.3111 - val_accuracy: 0.9152
Epoch 10/10
1719/1719 - 4s - loss: 0.3251 - accuracy: 0.9077 - val_loss: 0.3016 - 

In [39]:
model.save("mnist.h5")

#Transfer Learning##

In [40]:
pretrained_mnist_model = tf.keras.models.load_model("mnist.h5")

In [41]:
pretrained_mnist_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 300)               235500    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
________________________________________________

In [42]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten_1: True
dense_4: True
leaky_re_lu_2: True
dense_5: True
leaky_re_lu_3: True
dense_6: True


In [43]:
for layer in pretrained_mnist_model.layers[:-1]:
  layer.trainable = False
  print(f"{layer.name}: {layer.trainable}")

flatten_1: False
dense_4: False
leaky_re_lu_2: False
dense_5: False
leaky_re_lu_3: False


In [44]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten_1: False
dense_4: False
leaky_re_lu_2: False
dense_5: False
leaky_re_lu_3: False
dense_6: True


In [45]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]
new_model = tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(tf.keras.layers.Dense(1,activation="sigmoid"))

In [46]:
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 300)               235500    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 265,701
Trainable params: 101
Non-trainable params: 265,600
______________________________________________

In [47]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [48]:
y_train.shape

(55000,)

In [49]:
np.where(y_train%2 == 0, 1,0)

array([0, 0, 1, ..., 0, 1, 1])

In [50]:
y_train[0],y_train[-1]

(7, 8)

In [51]:
def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label%2 ==0, 1,0)
  return labels  

In [52]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train,y_test,y_valid])

In [53]:
np.unique(y_train_bin)

array([0, 1])

In [54]:
new_model.compile(loss = "binary_crossentropy",
                  optimizer = tf.keras.optimizers.SGD(lr=1e-3),
                  metrics="accuracy")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [55]:
history = new_model.fit(X_train,y_train_bin,epochs=10,
                        validation_data=(X_valid,y_valid_bin),verbose=2)

Epoch 1/10
1719/1719 - 3s - loss: 0.4631 - accuracy: 0.7917 - val_loss: 0.3569 - val_accuracy: 0.8564
Epoch 2/10
1719/1719 - 3s - loss: 0.3552 - accuracy: 0.8486 - val_loss: 0.3265 - val_accuracy: 0.8626
Epoch 3/10
1719/1719 - 3s - loss: 0.3365 - accuracy: 0.8555 - val_loss: 0.3126 - val_accuracy: 0.8686
Epoch 4/10
1719/1719 - 3s - loss: 0.3262 - accuracy: 0.8603 - val_loss: 0.3041 - val_accuracy: 0.8712
Epoch 5/10
1719/1719 - 3s - loss: 0.3191 - accuracy: 0.8631 - val_loss: 0.2979 - val_accuracy: 0.8740
Epoch 6/10
1719/1719 - 3s - loss: 0.3136 - accuracy: 0.8659 - val_loss: 0.2929 - val_accuracy: 0.8756
Epoch 7/10
1719/1719 - 3s - loss: 0.3092 - accuracy: 0.8683 - val_loss: 0.2889 - val_accuracy: 0.8800
Epoch 8/10
1719/1719 - 3s - loss: 0.3055 - accuracy: 0.8699 - val_loss: 0.2856 - val_accuracy: 0.8826
Epoch 9/10
1719/1719 - 3s - loss: 0.3022 - accuracy: 0.8720 - val_loss: 0.2829 - val_accuracy: 0.8844
Epoch 10/10
1719/1719 - 3s - loss: 0.2994 - accuracy: 0.8733 - val_loss: 0.2798 - 

In [56]:
new_model.evaluate(X_test,y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.2918 - accuracy: 0.8764


[0.2917536199092865, 0.8763999938964844]

In [57]:
X_new = X_test[:3]

y_test[:3],y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [60]:
new_model.predict(X_new)

array([[0.02084452],
       [0.9682479 ],
       [0.11970055]], dtype=float32)

In [61]:
np.round(new_model.predict(X_new))

array([[0.],
       [1.],
       [0.]], dtype=float32)